<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-6/notebooks/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Setup

In [ ]:
%pip install ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


# Imports

In [ ]:
import os
import sys
import json
import pandas as pd
from itertools import combinations
import math
import statistics
import numpy as np
import warnings

from pprint import pprint

warnings.simplefilter("ignore")

%load_ext autotime

time: 504 µs (started: 2024-04-21 06:11:35 +00:00)


# Configuration

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/hw6")


class Path:
    current_dir = os.getcwd()
    data_dir = os.path.join(current_dir, "data")
    input_csv_file = os.path.join(data_dir, "hw6_clustering.txt")

    output_dir = os.path.join(current_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    task_output_file = os.path.join(output_dir, "task_op.txt")
    task_output_ref_file = os.path.join(output_dir, "task_ref_op.txt")

time: 479 ms (started: 2024-04-21 06:11:35 +00:00)


# Task 1:  Bradley-Fayyad-Reina (BFR) algorithm
In BFR, there are three sets of points that you need to keep track of:

**Discard set (DS), Compression set (CS), Retained set (RS)**

For each cluster in the DS and CS, the cluster is summarized by:
- `N`: The number of points
- `SUM`: the sum of the coordinates of the points
- `SUMSQ`: the sum of squares of coordinates


## Ref

In [ ]:
import sys
import time
from sklearn.cluster import KMeans
import numpy as np

np.random.seed(37)

if __name__ == '__main__':
    input_path = Path.input_csv_file
    n_cluster = 5
    output_path = Path.task_output_ref_file
    s_t = time.time()

    rs = set()
    ds_dict = {}
    ds_c_dict = {}
    ds_d_dict = {}
    ds_point_dict = {}
    cs_dict = {}
    cs_c_dict = {}
    cs_d_dict = {}
    cs_point_dict = {}
    D = 0

    with open(input_path, "r") as f:
        datas = np.array(f.readlines())

    npdata = []
    for data in datas:
        temp = np.array(data.strip('\n').split(','))
        npdata.append(temp)
    npdata = np.array(npdata).astype(np.float64)

    #step1
    np.random.shuffle(npdata)
    npdata = np.array_split(npdata, 5)
    data1 = npdata[0]

    #step2
    kmeans1 = KMeans(n_clusters=5 * n_cluster).fit(npdata[0][:, 2:])

    #step3
    clusters = {}
    for idx, cid in enumerate(kmeans1.labels_):
        if cid not in clusters.keys():
            clusters[cid] = []
        clusters[cid].append(idx)
    rs = set()
    #print(len(data1))
    for idx in clusters.values():
        if len(idx) == 1:
            #print(idx)
            rs.add(idx[0])
    ds = np.delete(data1, list(rs), axis=0)
    #print(len(data1))
    #print(len(ds))

    #step4
    kmeans2 = KMeans(n_clusters=n_cluster).fit(ds[:, 2:])
    clusters = {}
    for idx, cid in enumerate(kmeans2.labels_):
        if cid not in clusters.keys():
            clusters[cid] = []
        clusters[cid].append(idx)

    #step5
    for cid, idx in clusters.items():
        n = len(idx)
        ds_feature = ds[idx, 2:]
        SUM = np.sum(ds_feature, axis=0)
        SUMSQ = np.sum(np.square(ds_feature), axis=0)
        ds_dict[cid] = []
        ds_dict[cid].append(n)
        ds_dict[cid].append(SUM)
        ds_dict[cid].append(SUMSQ)

        points = np.array(ds[idx, 0]).astype(int).tolist()
        ds_point_dict[cid] = points

        centroid = SUM / n
        ds_c_dict[cid] = centroid

        d = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))
        ds_d_dict[cid] = d

    #step6
    data_rs = data1[list(rs), :]
    if len(rs) >= 5 * n_cluster:
        kmeans_temp = KMeans(n_clusters=5 * n_cluster).fit(data_rs[:, 2:])
        clusters = {}
        for idx, cid in enumerate(kmeans_temp.labels_):
            if cid not in clusters.keys():
                clusters[cid] = []
            clusters[cid].append(idx)
        rs = set()
        #print(len(data1))
        for idx in clusters.values():
            if len(idx) == 1:
                #print(idx)
                rs.add(idx[0])

        for cid, idx in clusters.items():
            if len(idx) > 1:
                n = len(idx)
                cs_feature = data_rs[idx, 2:]
                SUM = np.sum(cs_feature, axis=0)
                SUMSQ = np.sum(np.square(cs_feature), axis=0)
                cs_dict[cid] = []
                cs_dict[cid].append(n)
                cs_dict[cid].append(SUM)
                cs_dict[cid].append(SUMSQ)

                points = np.array(data_rs[idx, 0]).astype(int).tolist()
                cs_point_dict[cid] = points

                centroid = SUM / n
                cs_c_dict[cid] = centroid

                d = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))
                cs_d_dict[cid] = d
    #print(cs_dict)
    with open(output_path, "w") as f:
        f.write('The intermediate results:\n')
        num_ds = 0
        num_cs = 0
        for value in ds_dict.values():
            num_ds += value[0]
        for value in cs_dict.values():
            num_cs += value[0]
        result_str = 'Round 1: ' + str(num_ds) + ',' + str(len(cs_dict)) + ',' + str(num_cs) + ',' + str(len(rs)) + '\n'
        f.write(result_str)

    #step7-12
    D = 2 * np.sqrt(data1.shape[1] - 2)
    for i in range(2, 6):
        #step7
        for idx, value in enumerate(npdata[i - 1]):
            data = value[2:]
            #step8
            maxx = float('inf')
            cluster = -1
            for cid, values in ds_dict.items():
                mahalanobis_dis = np.sqrt(np.sum(np.square(np.divide(np.subtract(data, ds_c_dict[cid]), ds_d_dict[cid])), axis=0))
                if mahalanobis_dis < maxx:
                    maxx = mahalanobis_dis
                    cluster = cid

            if maxx < D and cluster != -1:
                n = ds_dict[cluster][0] + 1
                SUM = np.add(ds_dict[cluster][1], data)
                SUMSQ = np.add(ds_dict[cluster][2], np.square(data))
                ds_dict[cluster] = []
                ds_dict[cluster].append(n)
                ds_dict[cluster].append(SUM)
                ds_dict[cluster].append(SUMSQ)

                centroid = SUM / n
                ds_c_dict[cluster] = centroid

                d = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))
                ds_d_dict[cluster] = d

                ds_point_dict[cluster].append(int(value[0]))
            else:
                #step9
                maxx = float('inf')
                cluster = -1
                for cid, values in cs_dict.items():
                    mahalanobis_dis = np.sqrt(np.sum(np.square(np.divide(np.subtract(data, cs_c_dict[cid]), cs_d_dict[cid])), axis=0))
                    if mahalanobis_dis < maxx:
                        maxx = mahalanobis_dis
                        cluster = cid
                if maxx < D and cluster != -1:
                    n = cs_dict[cluster][0] + 1
                    SUM = np.add(cs_dict[cluster][1], data)
                    SUMSQ = np.add(cs_dict[cluster][2], np.square(data))
                    cs_dict[cluster] = []
                    cs_dict[cluster].append(n)
                    cs_dict[cluster].append(SUM)
                    cs_dict[cluster].append(SUMSQ)

                    centroid = SUM / n
                    cs_c_dict[cluster] = centroid

                    d = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))
                    cs_d_dict[cluster] = d

                    cs_point_dict[cluster].append(int(value[0]))
                else:
                    #step10
                    rs.add(idx)
        #step11
        data_rs = npdata[i - 1][list(rs), :]
        if len(rs) >= 5 * n_cluster:
            kmeans_temp = KMeans(n_clusters=5 * n_cluster).fit(data_rs[:, 2:])
            clusters = {}
            for idx, cid in enumerate(kmeans_temp.labels_):
                if cid not in clusters.keys():
                    clusters[cid] = []
                    clusters[cid].append(idx)
            rs = set()
            #print(len(data1))
            for idx in clusters.values():
                if len(idx) == 1:
                    #print(idx)
                    rs.add(idx[0])

            for cid, idx in clusters.items():
                if len(idx) > 1:
                    n = len(idx)
                    cs_feature = data_rs[idx, 2:]
                    SUM = np.sum(cs_feature, axis=0)
                    SUMSQ = np.sum(np.square(cs_feature), axis=0)
                    cs_dict[cid] = []
                    cs_dict[cid].append(n)
                    cs_dict[cid].append(SUM)
                    cs_dict[cid].append(SUMSQ)

                    points = np.array(data_rs[idx, 0]).astype(int).tolist()
                    cs_point_dict[cid] = points

                    centroid = SUM / n
                    cs_c_dict[cid] = centroid

                    d = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))
                    cs_d_dict[cid] = d

        #step12
        new_dict = {}
        for cid1 in cs_dict.keys():
            cluster = -1
            for cid2 in cs_dict.keys():
                if cid1 != cid2:
                    mahalanobis_dis1 = np.sqrt(np.sum(np.square(np.divide(np.subtract(cs_c_dict[cid1], cs_c_dict[cid2]), cs_d_dict[cid2], out=np.zeros_like(np.subtract(cs_c_dict[cid1], cs_c_dict[cid2])), where=cs_d_dict[cid2] != 0)), axis=0))
                    mahalanobis_dis2 = np.sqrt(np.sum(np.square(np.divide(np.subtract(cs_c_dict[cid2], cs_c_dict[cid1]), cs_d_dict[cid1], out=np.zeros_like(np.subtract(cs_c_dict[cid2], cs_c_dict[cid1])), where=cs_d_dict[cid1] != 0)), axis=0))
                    mahalanobis_dis = min(mahalanobis_dis1, mahalanobis_dis2)
                    if mahalanobis_dis < D:
                        D = mahalanobis_dis
                        cluster = cid2
            new_dict[cid1] = cluster
        for cid1, cid2 in new_dict.items():
            if cid1 in cs_dict and cid2 in cs_dict:
                if cid1 != cid2:
                    n = cs_dict[cid1][0] + cs_dict[cid2][0]
                    SUM = np.add(cs_dict[cid1][1], cs_dict[cid2][1])
                    SUMSQ = np.add(cs_dict[cid1][2], cs_dict[cid2][2])
                    cs_dict[cid2][0] = n
                    cs_dict[cid2][1] = SUM
                    cs_dict[cid2][2] = SUMSQ

                    centroid = SUM / n
                    cs_c_dict[cid2] = centroid

                    d = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))
                    cs_d_dict[cid2] = d

                    cs_point_dict[cid2].extend(cs_point_dict[cid1])

                    cs_dict.pop(cid2)
                    cs_c_dict.pop(cid2)
                    cs_d_dict.pop(cid2)
                    cs_point_dict.pop(cid2)
        #step13 last iteration
        if i == 5:
            new_dict = {}
            for cid1 in cs_dict.keys():
                cluster = -1
                for cid2 in ds_dict.keys():
                    if cid1 != cid2:
                        mahalanobis_dis1 = np.sqrt(np.sum(np.square(np.divide(np.subtract(cs_c_dict[cid1], ds_c_dict[cid2]), ds_d_dict[cid2], out=np.zeros_like(np.subtract(cs_c_dict[cid1], ds_c_dict[cid2])), where=ds_d_dict[cid2] != 0)), axis=0))
                        mahalanobis_dis2 = np.sqrt(np.sum(np.square(np.divide(np.subtract(ds_c_dict[cid2], cs_c_dict[cid1]), cs_d_dict[cid1], out=np.zeros_like(np.subtract(ds_c_dict[cid2], cs_c_dict[cid1])), where=cs_d_dict[cid1] != 0)), axis=0))
                        mahalanobis_dis = min(mahalanobis_dis1, mahalanobis_dis2)
                        if mahalanobis_dis < D:
                            D = mahalanobis_dis
                            cluster = cid2
                new_dict[cid1] = cluster
            for cid1, cid2 in new_dict.items():
                if cid1 in cs_dict and cid2 in ds_dict:
                    if cid1 != cid2:
                        n = cs_dict[cid1][0] + ds_dict[cid2][0]
                        SUM = np.add(cs_dict[cid1][1], ds_dict[cid2][1])
                        SUMSQ = np.add(cs_dict[cid1][2], ds_dict[cid2][2])
                        ds_dict[cid2][0] = n
                        ds_dict[cid2][1] = SUM
                        ds_dict[cid2][2] = SUMSQ

                        centroid = SUM / n
                        ds_c_dict[cid2] = centroid

                        d = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))
                        ds_d_dict[cid2] = d

                        ds_point_dict[cid2].extend(cs_point_dict[cid1])

                        cs_dict.pop(cid1)
                        cs_c_dict.pop(cid1)
                        cs_d_dict.pop(cid1)
                        cs_point_dict.pop(cid1)

        with open(output_path, "a") as f:
            num_ds = 0
            num_cs = 0
            for value in ds_dict.values():
                num_ds += value[0]
            for value in cs_dict.values():
                num_cs += value[0]
            result_str = 'Round ' + str(i) + ': ' + str(num_ds) + ',' + str(len(cs_dict)) + ',' + str(num_cs) + ',' + str(len(rs)) + '\n'
            f.write(result_str)
    if len(rs) > 0:
        data_rs = npdata[4][list(rs), 0]
        rs = set([int(n) for n in data_rs])
    result = {}
    for cid in ds_dict.keys():
        for point in ds_point_dict[cid]:
            result[point] = cid
    for cid in cs_dict.keys():
        for point in cs_point_dict[cid]:
            result[point] = -1
    for point in rs:
        result[point] = -1

    with open(output_path, "a") as f:
        f.write('\n')
        f.write('The clustering results:\n')
        order_dict = sorted(result.keys(), key=int)
        for point in order_dict:
            f.write(str(point) + ',' + str(result[point]) + '\n')

    e_t = time.time()
    print('Duration: ', e_t - s_t)

Duration:  26.34880781173706
time: 26.4 s (started: 2024-04-21 06:18:11 +00:00)


In [ ]:
%cat output/task_ref_op.txt | head -n 20

The intermediate results:
Round 1: 64462,0,0,1
Round 2: 128904,0,0,22
Round 3: 193341,0,0,25
Round 4: 257793,0,0,25
Round 5: 322236,0,0,25

The clustering results:
0,1
1,1
2,2
3,1
4,0
5,0
6,3
7,1
8,0
9,0
10,1
11,0
time: 107 ms (started: 2024-04-21 06:18:06 +00:00)


## Version 4

In [28]:
import sys
import time
import traceback

import numpy as np
from sklearn.cluster import KMeans


LARGE_K_FACTOR = 5


def read_data(path):
    data = []
    with open(path, "r") as f:
        for line in f:
            row = line.strip().split(",")
            data.append([float(value) for value in row])
    return np.array(data)


def get_random_splits(arr: np.array):
    """
    Splits the input array into 5 roughly equal parts after shuffling.
    """
    np.random.shuffle(arr)
    return np.array_split(arr, 5)


def cluster_points(data: np.array, n_cluster: int, factor: int):
    """
    Clusters the data points using KMeans algorithm.

    Args:
        data (np.array): The input data array containing points.
        n_cluster (int): The desired number of clusters.

    Returns:
        Dict[int, List[int]]: A dictionary where keys represent cluster IDs and values contain the indices of points
        assigned to each cluster.
    """
    model = KMeans(n_clusters=factor*n_cluster)
    model.fit(data[:, 2:])

    clusters = {}
    for i, cluster_id in enumerate(model.labels_):
        if cluster_id not in clusters.keys():
            clusters[cluster_id] = []
        clusters[cluster_id].append(i)
    return clusters


def process_cluster_data(clusters, data, track_set, check_len=False):
    for cluster_id, idx in clusters.items():
        n = len(idx)
        if check_len and n <= 1:
            continue

        feats = data[idx, 2:]
        row_ids = data[idx, 0]

        SUM = np.sum(feats, axis=0)
        SUMSQ = np.sum(np.square(feats), axis=0)

        ctr = SUM / n
        pts = row_ids.astype(int).tolist()
        var = np.sqrt(np.subtract(SUMSQ / n, np.square(ctr)))

        track_set["smr"][cluster_id] = [n, SUM, SUMSQ]
        track_set["pts"][cluster_id] = pts
        track_set["ctr"][cluster_id] = ctr
        track_set["var"][cluster_id] = var

    return track_set


def m_dist_pt(feat, ctr, var):
    diff = feat - ctr
    z = np.square(diff / var)
    return np.sqrt(np.sum(z))


def m_dist_clusters(ctr1, ctr2, var):
    diff = ctr1 - ctr2
    z = np.square(diff / var)
    return np.sqrt(np.sum(z))


def reassign_cluster(feat, t_set):
    max_dist = np.inf
    cluster = -1
    for cluster_id, values in t_set["smr"].items():
        ctr = t_set["ctr"][cluster_id]
        var = t_set["var"][cluster_id]

        m_dist = m_dist_pt(feat, ctr, var)
        if m_dist < max_dist:
            max_dist = m_dist
            cluster = cluster_id

    return max_dist, cluster


def update_cluster_summary(feat, t_set, cluster, row_id):
    n = t_set["smr"][cluster][0] + 1
    SUM = np.add(t_set["smr"][cluster][1], feat)
    SUMSQ = np.add(t_set["smr"][cluster][2], np.square(feat))

    ctr = SUM / n
    var = np.sqrt(np.subtract(SUMSQ / n, np.square(ctr)))

    t_set["smr"][cluster] = [n, SUM, SUMSQ]
    t_set["ctr"][cluster] = ctr
    t_set["var"][cluster] = var
    t_set["pts"][cluster].append(row_id)

    return t_set


def merge_clusters(t_set1, t_set2, threshold_dist, inverse=False):
    new_assignment = {}
    for c1 in t_set1["smr"].keys():
        cluster = -1
        for c2 in t_set2["smr"].keys():
            if c1 != c2:
                m_dist1 = m_dist_clusters(t_set1["ctr"][c1], t_set2["ctr"][c2], t_set2["var"][c2])
                m_dist2 = m_dist_clusters(t_set2["ctr"][c2], t_set1["ctr"][c1], t_set1["var"][c1])
                m_dist = min(m_dist1, m_dist2)
                if m_dist < threshold_dist:
                    threshold_dist = m_dist
                    cluster = c2
        new_assignment[c1] = cluster

    for c1, c2 in new_assignment.items():
        if c1 in t_set1["smr"] and c2 in t_set2["smr"]:
            if c1 != c2:
                n = t_set1["smr"][c1][0] + t_set2["smr"][c2][0]
                SUM = np.add(t_set1["smr"][c1][1], t_set2["smr"][c2][1])
                SUMSQ = np.add(t_set1["smr"][c1][2], t_set2["smr"][c2][2])

                ctr = SUM / n
                var = np.sqrt(np.subtract(SUMSQ / n, np.square(ctr)))

                t_set2["smr"][c2] = [n, SUM, SUMSQ]
                t_set2["ctr"][c2] = ctr
                t_set2["var"][c2] = var
                t_set2["pts"][c2].extend(t_set1["pts"][c1])

                cluster_to_pop = c1 if inverse else c2

                t_set1["smr"].pop(cluster_to_pop)
                t_set1["ctr"].pop(cluster_to_pop)
                t_set1["var"].pop(cluster_to_pop)
                t_set1["pts"].pop(cluster_to_pop)

    return t_set1, t_set2


def update_round_results(
    round_res:list, round_num: int, ds_smr: dict, cs_smr: dict, r_set: set
):
    """
    Write intermediate clustering results to a file.

    Parameters:
        output_path (str): The path to the output file.
        round_num (int): The current round number of clustering.
        ds_smr (dict): A dictionary containing parameters for the Discard Set clusters.
        cs_params_dict (dict): A dictionary containing parameters for the Compression Set clusters.
        r_set (set): A set containing isolated points in the Retained Set.
    """
    num_ds = sum(value[0] for value in ds_smr.values())
    num_cs = sum(value[0] for value in cs_smr.values())

    result_str = f"Round {round_num}: {num_ds},{len(cs_smr)},{num_cs},{len(r_set)}\n"
    round_res.append(result_str)

    return round_res


def save_results(round_results, final_results, output_path):
    with open(output_path, "w") as f:
        f.write('The intermediate results:\n')
        f.writelines(round_results)

        f.write('\n')
        f.write('The clustering results:\n')
        for k,v in final_results.items():
            f.write(f"{int(k)},{int(v)}\n")


def BFR(data: np.array, n_cluster: int, output_path: str):
    # Create Dictionaries to keep track of data
    # "smr": Summary statistics, "ctr": Centroids, "var": Variances, "pts": Data points indexes.
    d_set = {"smr": {}, "ctr": {}, "var": {}, "pts": {}} # Discarded Set
    c_set = {"smr": {}, "ctr": {}, "var": {}, "pts": {}} # Compressed Set

    # Store results
    round_res = []
    final_res = {}
    flag = False

    # Step 1: Load 20% of the data randomly
    data = get_random_splits(data)
    split1 = data[0]

    # Step 2: Run K-Means (e.g., from sklearn) with a large K (e.g., 5 times of the number of the input clusters)
    # on the data in memory using the Euclidean distance as the similarity measurement
    clusters = cluster_points(split1, n_cluster, LARGE_K_FACTOR)

    # Step 3: In the K-Means result from Step 2, move all the clusters that contain only one point to RS (outliers).
    r_set = {idx[0] for idx in clusters.values() if len(idx) == 1}
    ds_data = np.delete(split1, list(r_set), axis=0)

    # Step 4: Run K-Means again to cluster the rest of the data points with K = the number of input clusters.
    clusters = cluster_points(ds_data, n_cluster, 1)

    # Step 5: Use the K-Means result from Step 4 to generate the DS clusters (i.e., discard their points and
    # generate statistics).
    d_set = process_cluster_data(clusters, ds_data, d_set)

    # The initialization of DS has finished, so far, you have K numbers of DS clusters (from Step 5) and some
    # numbers of RS (from Step 3).
    rs_data = split1[list(r_set), :]
    if len(r_set) >= LARGE_K_FACTOR * n_cluster:
        clusters = cluster_points(rs_data, n_cluster, LARGE_K_FACTOR)
        r_set = {idx[0] for idx in clusters.values() if len(idx) == 1}
        c_set = process_cluster_data(clusters, rs_data, c_set, check_len=True)

    # Store intermediate results for 1st round
    round_res = update_round_results(round_res, 1, d_set["smr"], c_set["smr"], r_set)

    THRESHOLD_DIST = 2 * np.sqrt(split1.shape[1] - 2)

    # Repeat Step 7 - 12
    for round in range(2,6):
        # Step 7: Load another 20% of the data randomly.
        split = data[round - 1]

        for idx, value in enumerate(split):
            row_id = int(value[0])
            feat = value[2:]

            # Step 8: For the new points, compare them to each of the DS using the Mahalanobis Distance and assign them
            # to the nearest DS clusters if the distance is < 2 * sqrt(𝑑).
            max_dist, cluster = reassign_cluster(feat, d_set)
            if max_dist < THRESHOLD_DIST and cluster != -1:
                d_set = update_cluster_summary(feat, d_set, cluster, row_id)

            # Step 9: For the new points that are not assigned to DS clusters, using the Mahalanobis Distance and assign
            # the points to the nearest CS clusters if the distance is < 2 * sqrt(𝑑)
            else:
                max_dist, cluster = reassign_cluster(feat, c_set)
                if max_dist < THRESHOLD_DIST and cluster != -1:
                    c_set = update_cluster_summary(feat, c_set, cluster, row_id)

                # Step 10: For the new points that are not assigned to a DS cluster or a CS cluster, assign them to RS.
                else:
                    r_set.add(idx)

        # Step 11: Run K-Means on the RS with a large K (e.g., 5 times of the number of the input clusters) to generate
        # CS (clusters with more than one points) and RS (clusters with only one point).
        rs_split = split[list(r_set), :]
        if len(r_set) >= LARGE_K_FACTOR * n_cluster:
            clusters = cluster_points(rs_split, n_cluster, LARGE_K_FACTOR)
            r_set = {idx[0] for idx in clusters.values() if len(idx) == 1}
            c_set = process_cluster_data(clusters, rs_split, c_set, check_len=True)

        if flag:
            # Step 12: Merge CS clusters that have a Mahalanobis Distance < 2 * sqrt(𝑑).
            c_set, _ =  merge_clusters(c_set, c_set, THRESHOLD_DIST, inverse=False)

            if round == 5:
                c_set, d_set =  merge_clusters(c_set, d_set, THRESHOLD_DIST, inverse=True)

        # Store intermediate results for every round
        round_res = update_round_results(round_res, round, d_set["smr"], c_set["smr"], r_set)

    if len(r_set) > 0:
        rs_data = data[4][list(r_set), 0]
        r_set = set([int(n) for n in rs_data])

    # Process points in discarded set
    for cluster_id in d_set["smr"].keys():
        for pt in d_set["pts"][cluster_id]:
            final_res[pt] = cluster_id

    # Process points in compressed set
    for cluster_id in c_set["smr"].keys():
        for pt in c_set["pts"][cluster_id]:
            final_res[pt] = -1

    # Process points in retained set
    for pt in r_set:
        final_res[pt] = -1

    return round_res, final_res


def task(input_path: str, n_cluster: int, output_path: str):
    np.random.seed(37)
    start_time = time.time()
    try:
        data = read_data(input_path)

        round_results, final_results = BFR(data, n_cluster, output_path)

        save_results(round_results, final_results, output_path)
    except Exception:
        traceback.print_exc()

    execution_time = time.time() - start_time
    print(f"Duration: {execution_time}\n")


if __name__ == "__main__":
    # Check if correct number of command-line arguments are provided
    if len(sys.argv) != 4:
        print("Usage: python task.py <input_filename> <n_cluster> <output_filename>")
        sys.exit(1)

    # Parse command-line arguments
    input_path = sys.argv[1]
    n_cluster = int(sys.argv[2])
    output_path = sys.argv[3]

    # Call task1 function
    task(input_path, n_cluster, output_path)

# task(Path.input_csv_file, 5, Path.task_output_file)

['Round 1: 64462,0,0,1\n',
 'Round 2: 128904,0,0,22\n',
 'Round 3: 193341,10,32,15\n',
 'Round 4: 257793,10,32,25\n',
 'Round 5: 322253,6,15,19\n']
Duration: 29.49673867225647

time: 29.5 s (started: 2024-04-21 09:53:27 +00:00)


# THE END